In [1]:
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
import uuid
import json

# --- CONFIGURATION FRANÇAISE 🇫🇷 ---
EMBEDDING_MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"
COLLECTION_NAME = "royaume_du_muffin_6"
FICHIER_JSON = "base_de_donnees.json"

def load_and_simulate_data():
    
    with open(FICHIER_JSON, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return pd.DataFrame(data)


def create_embeddings_and_store(df):
    print("🤖 Chargement du modèle d'embedding multilingue...")
    model = SentenceTransformer(EMBEDDING_MODEL_NAME)

    df_copy = df.copy()
    df_copy = df_copy.fillna("")
    
    # On transforme la colonne 'ingredients' (liste) en texte (str)
    # On vérifie d'abord si c'est bien une liste pour éviter de faire planter le code
    df_copy['ingredients'] = df_copy['ingredients'].apply(
        lambda x: ", ".join(x) if isinstance(x, list) else x
    )
    # Concaténation Titre + Ingrédients pour la recherche
    documents = df_copy["text_for_embedding"].tolist()
    metadatas = df_copy.to_dict(orient='records')
    ids = [str(uuid.uuid4()) for _ in range(len(df_copy))]

    print("⚡ Vectorisation en cours...")
    embeddings = model.encode(documents).tolist()


    # Stockage ChromaDB
    client = chromadb.Client() # En mémoire pour le test
    try: client.delete_collection(name=COLLECTION_NAME)
    except: pass

    collection = client.create_collection(name=COLLECTION_NAME)
    collection.add(documents=documents, embeddings=embeddings, metadatas=metadatas, ids=ids)

    print(f"✅ Indexation terminée ! {collection.count()} recettes stockées.")
    return collection



c:\miniconda\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] La procédure spécifiée est introuvable'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
# --- TEST ---
if __name__ == "__main__":
    df = load_and_simulate_data()
    db = create_embeddings_and_store(df)

🤖 Chargement du modèle d'embedding multilingue...
⚡ Vectorisation en cours...
✅ Indexation terminée ! 375 recettes stockées.


In [3]:
# Test de recherche en français
query = "Je suis fan de pizza"
model = SentenceTransformer(EMBEDDING_MODEL_NAME)
results = db.query(query_embeddings=model.encode([query]).tolist(), n_results=6)
print(f"\n🔎 Question: '{query}'")

for i, metadata in enumerate(results['metadatas'][0]):
    print(f"👉 Top {i+1}: {metadata['titre']}")


🔎 Question: 'Je suis fan de pizza'
👉 Top 1: Pizza façon muffin
👉 Top 2: Muffins pizzas faciles
👉 Top 3: Muffins 'pizza'
👉 Top 4: Muffins burger à l'italienne
👉 Top 5: Muffin feuilleté tomate aubergine mozzarella
👉 Top 6: Muffins au fromage à raclette


In [4]:
def preparer_contexte(results):
    """Transforme les résultats de ChromaDB en texte lisible par le LLM"""
    context_parts = []
    for i in range(len(results['documents'][0])):
        titre = results['metadatas'][0][i]['titre']
        ingredients = results['metadatas'][0][i]['description'] 
        instructions = results['metadatas'][0][i]['instructions']
        
        recette_texte = f"RECETTE {i+1}: {titre}\nIngrédients: {ingredients}\nInstructions: {instructions}"
        context_parts.append(recette_texte)
        
    return "\n\n---\n\n".join(context_parts)

In [5]:
import ollama

def generer_reponse_rag(query, context):
    # Le prompt du prof
    system_prompt = f"""
    TU ES "CHEF MUFFIN", UN ASSISTANT CULINAIRE OBSESSIONNEL MAIS SYMPATHIQUE.
    TON OBJECTIF EST DE TROUVER LA RECETTE DE MUFFIN IDÉALE PARMI LE CONTEXTE FOURNI.

    ### TES DIRECTIVES (GUARDRAILS) :
    1. OBSESSION : Tu ne cuisines QUE des muffins. Si on te demande des lasagnes ou une pizza, REFUSE poliment avec humour.
    2. ANCRAGE : Utilise UNIQUEMENT les recettes fournies dans le bloc [CONTEXTE]. N'invente rien.
    3. LANGUE : Réponds toujours en français courant et appétissant.

    [CONTEXTE]
    {context}

    [QUESTION]
    {query}
    """

    # Appel au LLM
    response = ollama.chat(model='mistral', messages=[
        {'role': 'user', 'content': system_prompt},
    ])
    
    return response['message']['content']

In [ ]:
# Pour google colab
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
import uuid
from mistralai import Mistral


# configuration
# MISTRAL_API_KEY = "u05c6XjGJKcsviOcDlv0OjQSFhg8ztmY" 
EMBEDDING_MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"
COLLECTION_NAME = "muffin_colab_pro"

# Recherche dans la base de données
def create_db_on_colab(df):
    df_copy = df.copy().fillna("") 
    model = SentenceTransformer(EMBEDDING_MODEL_NAME)
    for col in df_copy.columns:
        df_copy[col] = df_copy[col].apply(lambda x: ", ".join(map(str, x)) if isinstance(x, list) else x)
    
    
    # Normalisation pour des scores propres
    embeddings = model.encode(df_copy["text_for_embedding"].tolist(), normalize_embeddings=True).tolist()
    
    client = chromadb.Client()
    try:
        client.delete_collection(name=COLLECTION_NAME)
    except:
        pass
    
    collection = client.create_collection(name=COLLECTION_NAME)
    
    collection.add(
        documents=df_copy["text_for_embedding"].tolist(),
        embeddings=embeddings,
        metadatas=df_copy.to_dict(orient='records'),
        ids=[str(uuid.uuid4()) for _ in range(len(df_copy))]
    )
    return collection, model

# Génération de texte avec Mistral API
def generer_reponse_chef(query, results):
    client = Mistral(api_key=MISTRAL_API_KEY)
    
    # On construit le contexte à partir des résultats de ChromaDB
    contexte = "\n".join([f"- {m['titre']}: {m['description']}" for m in results['metadatas'][0]])
    

    # Instructions pour mon prompt
    prompt = f"""TU ES CHEF MUFFIN, UN ASSISTANT CULINAIRE OBSESSIONNEL MAIS SYMPATHIQUE.
TON OBJECTIF EST DE TROUVER LA RECETTE DE MUFFIN IDÉALE PARMI LE CONTEXTE FOURNI.

### TES DIRECTIVES (GUARDRAILS) :
1. OBSESSION : Tu ne cuisines QUE des muffins. Si on te demande des lasagnes ou une pizza, REFUSE poliment avec humour.
2. ANCRAGE : Utilise UNIQUEMENT les recettes fournies dans le bloc [CONTEXTE]. N'invente rien.
3. LANGUE : Réponds toujours en français courant et appétissant.
4. CORRECTION : si l'utilisateur te demande de cuisiner avec des choses qui ne sont pas des aliments, réponds lui avec humour que tu n'es pas mécanicien, ou magicien etc... 
5. Il y a plusieurs cas, si l'utilistaeur te donne des ingrédients/à une requête qui correspond très bien avec l'une des 3 recettes de results, alors ne renvoit que cette recette à l'utilisateur,
si les 3 propositions sont proches mais ne correspondent pas exactement, dis à l'utilisateur que tu n'as pas en stock une recette qui correspond parfaitement à ses attentes mais propose
lui les trois recettes en suggestions, pour que ça l'inspire ! Attention, ces recettes doivent quand même contejnir au moins l'un des ingrédient demandé, ou bien être dans la même famille d'aliment :
par exemple si je demande courgettes il me propose au moins un muffin avec un autre légume. Si les 3 propositions n'ont rien à voir alors ne rien renvoyer. 
Si l'utilisateur te donne des ingrédients pour une recette salée, ne lui propose pas les recettes sucrées.

Dans tous les cas, réponds toujours avec bonne humeur, entrain et humour ! Tu es un fan inconditionnel de muffins.

[CONTEXTE]
{contexte}
[QUESTION]
{query} """
    chat_response = client.chat.complete(
          model="mistral-small-latest", # Modèle équilibré et efficace
          messages=[
              {
                  "role": "user",
                  "content": prompt,
              },
          ]
      )
      
    return chat_response.choices[0].message.content
# TEST
# Charge la base de données
df = pd.read_json('/base_de_donnees.json') 

# Simulation d'une recherche
collection, embedder = create_db_on_colab(df)
query = "j'aime beaucoup la pizza" # Entrer la requête
res = collection.query(query_embeddings=embedder.encode([query], normalize_embeddings=True).tolist(), n_results=3) # On prend les 3 meilleurs résultats par ChromaDB
print(generer_reponse_chef(query, res))

In [ ]:
import streamlit as st
import random
import time

st.write("Streamlit loves LLMs! 🤖 [Build your own chat app](https://docs.streamlit.io/develop/tutorials/llms/build-conversational-apps) in minutes, then make it powerful by adding images, dataframes, or even input widgets to the chat.")

st.caption("Note that this demo app isn't actually connected to any LLMs. Those are expensive ;)")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = [{"role": "assistant", "content": "Let's start chatting! 👇"}]

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Accept user input
if prompt := st.chat_input("What is up?"):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})
    # Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)

    # Display assistant response in chat message container
    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        full_response = ""
        assistant_response = random.choice(
            [
                "Hello there! How can I assist you today?",
                "Hi, human! Is there anything I can help you with?",
                "Do you need help?",
            ]
        )
        # Simulate stream of response with milliseconds delay
        for chunk in assistant_response.split():
            full_response += chunk + " "
            time.sleep(0.05)
            # Add a blinking cursor to simulate typing
            message_placeholder.markdown(full_response + "▌")
        message_placeholder.markdown(full_response)
    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant", "content": full_response})
